In [1]:
import librosa
import librosa.display

import numpy as np
import matplotlib.pyplot as plt

# from past.utils import old_div

y, sr = librosa.load("D:\Drum\HiHat\hat 2 SMOKE.wav")
#y, sr = librosa.load("D:\Drum\HiHat\!Hat 39.wav")
#y, sr = librosa.load('/Users/cooky/HDD/Drum/Dilla Drums/!Snare 170.wav')
#y, sr = librosa.load('/Users/cooky/HDD/Drum/illmind blap kit 7/KICK_FUELO.wav')
#y, sr = librosa.load('/Users/cooky/HDD/Drum/Dilla Drums/!Snare 170.wav')

#print(y.shape)

yt, index = librosa.effects.trim(y=y, top_db=30)



yt = librosa.util.normalize(yt)

In [2]:
CNN_INPUT_SIZE = (128, 20)
DEFAULT_SR = 22050

In [3]:
import numpy as np

def resize_axis(array, N):
    if(array.shape[1] > N):
        resized = array[:,:N]
    else:
        resized = np.lib.pad(array, ((0,0),(0,N - array.shape[1])),\
            'constant', constant_values=(np.min(array)))
    return resized

In [4]:
def extract_cnn_input(raw_audio):
    frame_length = min(2048, len(raw_audio))
    mel_spec = librosa.core.power_to_db(librosa.feature.melspectrogram(
        y=raw_audio, sr=DEFAULT_SR, n_fft=frame_length,
        hop_length=frame_length//4, n_mels=CNN_INPUT_SIZE[0])
    )
    # Truncate number of frames stored
    m = min(CNN_INPUT_SIZE[1], mel_spec.shape[1])
    N =20
    mell = resize_axis(mel_spec[:, 0:m], N)
    return mell

---

In [ ]:
MIN_LEN = 8
MAX_LEN = 14

def len_profit(y):
    return True if len(y) > MIN_LEN else False

def chg_len(y):
    if len(y) > MAX_LEN:
        return y[:MAX_LEN]
    else:
        for i in range(len(y), MAX_LEN):
            y.append(0)
        return y
    

In [22]:
#for y in yt:
for path in range(1):
    # load
    y, sr = librosa.load(path)
    # trim
    yt, index = librosa.effects.trim(y=y, top_db=30)
    # normalize
    yt = librosa.util.normalize(yt)
    
    imsi_zcr = librosa.feature.zero_crossing_rate(yt)
    
    if len_profit(imsi_zcr):
        # Duration
        dur = librosa.get_duration(yt)
        # Zero Crossing Rate
        zcr = librosa.feature.zero_crossing_rate(yt)
        # Spectral Flatness
        flt = librosa.feature.spectral_flatness(yt)
        # Spectral Bandwidth
        bdw = librosa.feature.spectral_bandwidth(yt)
        # Spectral RollOff
        rlf = librosa.feature.spectral_rolloff(yt, roll_percent=0.8)
        # --------
        # Mel Spectogram
        # mel = extract_cnn_input(yt)
        # Constant-Q Power
        # cqt = np.abs(librosa.cqt(yt))
        # --------
        # Concat
        feature = np.concatenate(dur, zcr, flt, bdw, rlf)
    

-----

In [39]:
import pandas as pd

sound_feature = pd.DataFrame(columns=['Name', 'kit', 'len', 'Duration', \
    'ZCR', 'Bandwidth', 'Rolloff', 'Mel', 'cqt', \
        'Mel', 'CQT'])